# Markov decision processes and the Bellman Equations
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eleni-vasilaki/rl-notes/blob/main/notebooks/05_mdp.ipynb)

We have previously explored how to estimate expectations of immediate rewards through the concept of **bandits**. However, these algorithms cannot adequately handle scenarios where rewards are obtained only after a sequence of actions. In such cases, the reward associated with a particular action is delayed, occurring only at some future point. Consider, for instance, the game of chess: winning typically follows a series of strategic moves. While each move is chosen to be optimal at that moment, most moves have only an indirect connection to the eventual reward. Such scenarios are more general than bandit problems and thus require a more comprehensive framework. Specifically, we need a structured representation that includes the possible states of the environment, how these states are interconnected, the available actions, and the probabilities associated with transitioning from one state to another given an action. This structured representation is known as a **Markov Decision Process** (MDP). Understanding MDPs is crucial because they serve as the theoretical foundation for many core reinforcement learning algorithms. In the following sections, we will explore MDPs and the associated Bellman Equation, a fundamental result underpinning Temporal Difference Learning algorithms designed to handle **future rewards**.

## Markov Decision Process

In a Markov Decision Process (MDP), the concept of a ‘state’ represents the situation or position in which the system or agent finds itself within the environment. It encapsulates all relevant information required for decision-making. The agent selects actions based on the current state, with the aim of maximising some notion of cumulative reward over time.

Diagrams commonly used to represent MDPs depict states as circles and actions as smaller solid circles. Arrows extending from these action circles indicate possible transitions to subsequent states. Importantly, when taking an action, the probabilities of transitioning to the possible next states must sum exactly to 1.


### Example

Consider an analogue safe. For simplicity, assume the dial has only two numbers (0 and 1). This would obviously be an insecure safe, but our goal is simply to illustrate a basic deterministic MDP scenario.

We start from an initial state $S_0$. At each step, we can turn the dial to either 0 or 1. If we turn it to the correct first number, we move from $S_0$ to state $S_1$. If we choose the incorrect number, we move to $S_2$. From $S_1$, choosing the correct second number moves us into the rewarded terminal state $TR$, the safe opens and we receive reward 1. Any incorrect choice leads to an unrewarded terminal state, again denoted as $T$, with reward 0.

Similarly, state $S_2$ (which we reach after an incorrect first choice) always leads to an unrewarded terminal state, regardless of the next action. Each intermediate step gives a reward of 0. This deterministic setting means all transitions have probability 1.

We can summarise the MDP as:

- **States:**  
  - $S = \{ S_0, S_1, S_2, TR, T \}$  

- **Actions:**  
  - $A = \{ 0, 1 \}$  

- **Transition probabilities:**  
  All transitions are deterministic, meaning they happen with probability **1** for the correct next state:  
 
  - **From $S_0$:**
    - If the correct first number is chosen: $P(S_1 \mid S_0, \text{correct}) = 1.$
    - If the incorrect first number is chosen: $P(S_2 \mid S_0, \text{incorrect}) = 1.$
  - **From $S_1$:**
    - If the correct second number is chosen: $P(TR \mid S_1, \text{correct}) = 1.$
    - If the incorrect second number is chosen: $P(T \mid S_1, \text{incorrect}) = 1.$
  - **From $S_2$:**
    - Regardless of the action: $P(T \mid S_2, \cdot) = 1.$

- **Rewards:**  
  - The only reward is $R(S1,\text{correct},TR) = 1$ for reaching $TR$ by following the correct sequence:  $P(R,TR \mid S1,\text{correct})=1$
  - All other transitions give $R = 0$.  


<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/deterministic_mdp.png" width="800" />
</div>

# Exercise: Stochastic MDP Case

In the example of the safe, now assume that there is a problem with the mechanism, and 10% of the times, while you have selected the correct digit, the mechanism slips and an incorrect digit is internally selected. Update the diagram. **Hint:** Now, from every correct action, two arrows will be departing. You will need to use the symbol below, which denotes that by taking an action $a$ you may end up in many different states. The transition probabilities to these states, however, should sum up to 1.



<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/stochastic_state_action_state.png" width="400" />
</div>


<details>
<summary>Show Solution</summary>
<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/stochastic_mdp.png" width="800" />
</div>
</details>

## Markovian Property

As seen before, an MDP consists of the following key components:
- A set of states $ S $ representing all possible situations the system can be in.
- A set of actions $ A $ available to the agent.
- A transition probability function $ P(s' \mid s, a) $ that defines the likelihood of moving to state $ s' $ after taking action $ a $ in state $ s $.
- A reward function $ R(s, a, s') $ specifying the immediate reward received when taking action $ a $ in state $ s $ and transitioning to state $ s' $. In some formulations this notation is simplified to $R(s, a)$, interpreting the reward as a consequence of the action. 

A key property of an MDP is the **Markovian property**, which states that the **next state and reward depend only on the current state and action** and are independent of past states and actions beyond the current time step. This property ensures that the probability of transitioning to any future state can be determined solely based on the current state and the action taken.

For a **Markov process** (without actions), this property is expressed as:

$$ 
P(S_{t+1} | S_t, S_{t-1}, ..., S_0) = P(S_{t+1} | S_t).
$$

This equation signifies that the probability of transitioning to the next state depends solely on the current state and is independent of all previous states.

In the **state-action formulation** of an MDP, the Markov property extends to include actions and rewards:

$$
P(S_{t+1}, R_{t+1} | S_t, A_t, S_{t-1}, A_{t-1}, \dots, S_0, A_0) = P(S_{t+1}, R_{t+1} | S_t, A_t).
$$

This states that the future state and reward depend only on the current state and action, not on any past states or actions.


### Example

Consider the simple act of rolling a fair six-sided die. The probability of rolling a six on any given toss is 1/6, regardless of the outcomes of previous rolls. This scenario demonstrates the Markovian property because the outcome of the current roll does not depend on the history of rolls. Each roll is an independent event, and the 'state' can be considered as the outcome of the current roll.

### State Augmentation

Transitioning to a more complex example, in the board game Monopoly, if we consider the state to be solely the location of a pawn on the board, this model would not be Markovian. The reason is that future actions and their outcomes (e.g., paying rent, buying properties) depend not only on the pawn's location but also on the player's available money or purchased property, which is a result of past actions.

However, by employing 'state augmentation', we can include the player's money and property ownership as part of the state. This augmented state now encapsulates both the pawn's location and the player's financial status, making the system Markovian. With this broader definition of state, the future actions and their outcomes depend solely on this augmented state, adhering to the Markovian property. The probability of any future transition is determined by the current augmented state and the actions taken, ensuring that past actions influence future possibilities only through their effect on the current state.

### Real Life and RL

In real life, and by extension in many practical issues Reinforcement Learning (RL) tries to solve, few processes are strictly Markovian. The past frequently has an effect on future actions, a reflection of the complex interdependencies and the accumulation of consequences over time. This reality makes state augmentation not just a useful but an essential technique in RL. By incorporating relevant historical information into the current state, state augmentation allows RL models to navigate and learn from environments where future states and rewards are not solely dependent on the immediate preceding state but also on the history leading up to it. This approach allows us to apply RL in complex, dynamic systems.

# Exercise

For each of the following scenarios, define an appropriate augmented state that makes the system Markovian.

1. **Stock Trading Strategy**: A trading algorithm decides to buy, sell, or hold based on the current price of a stock. However, the decision also depends on the trend over the last three days.

2. **Text-Based Game Navigation**: In a text-based adventure game, a player moves between rooms based on descriptions given by the system. The action to move back ("go back") to the previously visited room doesn't always lead to the same room, as the path is not linear but depends on the sequence of rooms visited. Assume that it depends on the previous two rooms.

3. **Weather Prediction**: A simple weather model predicts tomorrow’s temperature based only on today’s temperature. However, the real dynamics depend on both today’s temperature and the rate of change from the previous day.



<details>
<summary>Show Solution</summary>

1. **Stock Trading Strategy**: Augment the state to include the stock price at the current time step and the prices at the last three time steps. State = (PriceToday, PriceDayMinus1, PriceDayMinus2, PriceDayMinus3).

2. **Text-Based Game Navigation**: Augment the state to include a history of rooms visited, at least the last two rooms, to make "go back" deterministic. State = (CurrentRoom, PreviousRoom 1, PreviousRoom 2).

3. **Weather Prediction**: Augment the state to include today’s temperature and the difference between today’s and yesterday’s temperatures to capture the inertia of temperature changes. State = (TempToday, TempYesterday - TempTwoDaysAgo)

These examples define how history affects the current status of the agent, making state augmentation easy to formalise. In practical situations, however, it is not always clear how much of the past influences a problem, and when modelling, we have to make an assumption and test the results.

</details>


## Total Return

In the context of future rewards, we start from a state $ s $ with the goal of selecting actions that maximize the **expected return**, i.e., the average total reward across many episodes of playing the game. The agent selects actions $ a $ according to a policy $ \pi(a|s) $, which can be interpreted as a conditional probability. We previously introduced policies in the context of bandits, albeit without state dependence. Here, we extend these concepts to a multi-state scenario, where policies define probabilities conditioned on the state $ s $. 

In the simplest case, where the episode lasts for a fixed number of steps $ T $, the **total return** is defined as:

$$
G_t = R_{t+1} + R_{t+2} + R_{t+3} + \dots + R_T.
$$

However, in many scenarios, the number of steps is **not fixed**. We introduce a **discount factor** $ \gamma $, where $ 0 \leq \gamma < 1 $, and define the **discounted total return** as:

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots
$$

Since $ \gamma < 1 $, the term $ \gamma^{k-1} $ becomes **negligibly small** after sufficiently many steps. This means that optimisation occurs within a **finite window** of future actions, as distant rewards contribute less. The discount factor $ \gamma $ prioritises **immediate rewards** over distant future rewards, reflecting both the **uncertainty of long-term rewards** and the agent’s natural preference for immediate gains. This second equation applies even when the episode length is fixed, making it the **more general formulation**.


### Recursive Form for Total Return

We start with the definition of the return:

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots
$$

Factoring out $ \gamma $ from the remaining terms:

$$
G_t = R_{t+1} + \gamma \left( R_{t+2} + \gamma R_{t+3} + \gamma^2 R_{t+4} + \dots \right).
$$

Recognising that the expression in parentheses matches the definition of $ G_{t+1} $, we explicitly write:

$$
G_{t+1} = R_{t+2} + \gamma R_{t+3} + \gamma^2 R_{t+4} + \dots
$$

Replacing $ G_{t+1} $ in the original equation:

$$
G_t = R_{t+1} + \gamma G_{t+1}.
$$

This **recursive form** is fundamental in deriving the **Bellman equations**, which underpin key reinforcement learning algorithms.

### Discount factor and addiction
Models of MDP and Reinforcement Learning have been used to capture animal and human behaviour. For instance, addictive behaviour can be modelled with a small discount factor $\gamma$; the agent will prioritise actions that bring immediate rewards (e.g. drinking alcohol) over actions such as eating healthily and going to the gym that bring rewards in the future.

### Example

Imagine a garbage collecting robot at a junction with two paths: it can turn right ($G^{r}_0$) or turn left ($G^{l}_0$). We will calculate the total return, $G_0$, from the starting position to the end (Home) for both action choices, using a discount factor of $\gamma = 0.9$.

The sequences of rewards for turning right ($R^{r}$) or left ($R^{l}$) are represented as follows:

- **Right Path Rewards Sequence**: The sequence is $[+10, +10, +10, 0, 0, 0, 0, 0, 0, -50, 0, 0, 0]$. 
  - This corresponds to rewards $R^{r}_1 = +10$, $R^{r}_2 = +10$, $R^{r}_3 = +10$, with subsequent zero rewards until $R^{r}_{10} = -50$, followed by zero rewards to the end of the sequence. These are the rewards the robot collects at each step when it turns right. The $R^{r}_{10}=-50$ is because there is a hole on the road causing the robot some damage, but allowing it to continue to the Home terminal.

- **Left Path Rewards Sequence**: The sequence is $[+2, +2, +2, 0, 0, 0, 0, 0, 0, +2, +2, +2, +2 ]$.
  - This corresponds to $R^{l}_1 = +2$, $R^{l}_2 = +2$, $R^{l}_3 = +2$, with subsequent zero rewards until reaching $R^{l}_{10}$, from that point onwards it receives $+2$ rewards, including $R^{l}_{Home} = +2$. 

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/myopic.png" width="800" />
</div>

Note: In this diagram, we ommited denoting explicitly the action as $a$ for simplicity.

#### Calculation for $\gamma = 0.9$

To calculate the expected return $G_0$ from the starting position if the robot turns right ($G^{r}_0$) and left ($G^{l}_0$), we explicitly sum the rewards, weighted by $\gamma$ raised to the power of the step index:

- For the **Right Path** ($G^{r}_0$):

$$
G^{r}_0 = R^{r}_1 + \gamma R^{r}_2 + \gamma^2 R^{r}_3 + \ldots + \gamma^{9} R^{r}_{10} + \gamma^{10} R^{r}_{11} + \gamma^{11} R^{r}_{12} + \gamma^{12} R^{r}_{Home}
$$

After performing the calculation:

$$
G^{r}_0=10+ 0.9 \times 10+0.9^2 \times 10+0.9^9 \times (-50) \approx 7.73
$$

- For the **Left Path** ($G^{l}_0$):

$$
G^{l}_0 = R^{l}_1 + \gamma R^{l}_2 + \gamma^2 R^{l}_3 + \ldots + \gamma^{9} R^{l}_{10}+ \gamma^{10} R^{l}_{11} + \gamma^{11} R^{l}_{12} + \gamma^{12} R^{l}_{Home}
$$

After performing the calculation:

$$
G^{l}_0 =2+0.9 \times 2+ 0.9^2 \times 2 +0.9^9 \times 2+0.9^{10} \times 2+0.9^{11} \times 2 +0.9^{12} \times 2 \approx 8.08
$$

Therefore the path leading to the higest reward is the **Left Path**.

# Exercise

Study the scenario and calculations in the example above. Assume now that $\gamma = 0.5$. Which action seems better for the robot now, turning right or turning left? Calculate $G_0$ for both actions with $\gamma = 0.5$ and discuss which path the robot should choose to maximize its expected return.


<details>
<summary>Show Solution</summary>

With $\gamma = 0.5$, we calculate the expected returns for turning right and left again using the provided sequences. The calculation formula remains the same, but with a modified discount factor:

- For the **Right Path** ($G^{r}_0$) with $\gamma = 0.5$:

$$
G^{r}_0 = R^{r}_1 + \gamma R^{r}_2 + \gamma^2 R^{r}_3 + \ldots + \gamma^{9} R^{r}_{10} + \gamma^{10} R^{r}_{11} + \gamma^{11} R^{r}_{12} + \gamma^{12} R^{r}_{Home}
$$

Plugging in the values:

$$
G^{r}_0=10+ 0.5 \times 10+0.5^2 \times 10+0.5^9 \times (-50) \approx 17.40
$$


- For the **Left Path** ($G^{l}_0$) with $\gamma = 0.5$:

$$
G^{l}_0 = R^{l}_1 + \gamma R^{l}_2 + \gamma^2 R^{l}_3 + \ldots + \gamma^{9} R^{l}_{10}+ \gamma^{10} R^{l}_{11} + \gamma^{11} R^{l}_{12} + \gamma^{12} R^{l}_{Home}
$$

Plugging in the values:

$$
G^{l}_0 =2+0.5 \times 2+ 0.5^2 \times 2 +0.5^9 \times 2+0.5^{10} \times 2+0.5^{11} \times 2 +0.5^{12} \times 2 \approx 3.51
$$


Given the calculated expected returns, the robot should choose the **Right Path** when $\gamma = 0.5$ since it appears as if it offers a significantly higher expected return ($G^{r}_0 \approx 17.40$) compared to the **Left Path** ($G^{l}_0 \approx 3.51$). However, the agent will experience the $-50$ punishment when selecting the right path. Due to the small value of $\gamma$, the negative reward was heavily discounted and appeared insignificant to the agent. **Similarly**, the rewards near the Home terminal from the left path are also **discounted**. This demonstrates how a myopic choice of $\gamma$ may lead the agent to prefer sub-optimal actions, because rewards (and punishments) far in the future are devalued.

</details>

## Action-value function definition
In this framework, we expand the definition of the $q^*$ that we defined in the case of bandits and say:

$$ q^\pi(s, a) = \mathbb{E_{\pi}}[G_t | S_t = s, A_t = a],$$

which is the **action-value function** under policy $\pi(a|s)=p(a|s)$. We denote with small q the theoretical true value and with capital Q its estimate. For the moment we will focus on the true values $q$.

We will show that an equivalent expression for the action-value function is:

$$ q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1} + \gamma \; q^\pi(s', a') | S_t = s, A_t = a], $$

where starting from state $s$ and taking action $a$ under policy $\pi$ leads us to state $s'$, where in turn we take action $a'$. This is a fundamental relationship upon which we will build the first RL learning rule for future rewards, albeit not the typical form of the Bellman equation.

Here we should note that if $s'$ is terminal, we do not take another action but only observe the reward and stop there; in practice, $q^\pi(s', a')=0$. We also note that due to the stochasticity of the environment, there are many possible $s'$ states that we can reach with different probabilities $p(s'|s,a)$, for this we need to take an expectation.

## Proof

We start from the definition of the expected discounted return $G_t = R_{t+1}+ \gamma R_{t+2}+ \gamma^2 R_{t+3}+\ldots$ and write it as: $G_t = R_{t+1} + \gamma G_{t+1}.$ We recall the definition: $ q^\pi(s, a) = \mathbb{E}[G_t | S_t = s, A_t = a].$

Then we express $q^\pi(s, a)$ using $G_t$:

$$ q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1} + \gamma G_{t+1} | S_t = s, A_t = a].$$

We then apply the principle of linearity of expectation:

$$ q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1} | S_t = s, A_t = a] + \gamma \; \mathbb{E_{\pi}}[G_{t+1} | S_t = s, A_t = a].$$

For simplicity, we will define:

$$Part(1)= \mathbb{E_{\pi}}[R_{t+1} | S_t = s, A_t = a]$$

and

$$Part(2) =\mathbb{E_{\pi}}[G_{t+1} | S_t = s, A_t = a],$$

hence:

$$ q^\pi(s, a) = Part(1)+ \gamma Part(2).$$

We will leave $Part(1)$ aside and work on $Part(2)$. The key idea here is that if we condition the expectation on $G$ to the next state-action pair $s',a'$, we can replace it with $q^\pi(s',a')$. To achieve this, we will manipulate the expectation accordingly.

To condition on $S_{t+1}$ we use the law of total expectation:

$$ Part(2)=\; \sum_{s'} \; p(S_{t+1} = s' | S_t = s, A_t = a) ~ \mathbb{E_{\pi}}[G_{t+1} | S_t = s, A_t = a, S_{t+1} = s'].$$

By using the Markovian property, we can further simplify the expression:

$$ Part(2)= \sum_{s'} \; p(S_{t+1} = s' | S_t = s, A_t = a) \; \mathbb{E_{\pi}}[G_{t+1} | S_{t+1} = s'].$$

Then we condition to $A_{t+1}$ using again the law of total expectation with the view to form the term $q^\pi(s', a')$. In these calculations it is helpful to think of policy $\pi$ as a probability of choosing an action given the state:

$$ Part(2)= \sum_{s',a'} ~ \pi(a'|s') p(s' | s, a) \; \mathbb{E_{\pi}}[G_{t+1} | S_{t+1} = s', A_{t+1}=a'].$$

Since $\mathbb{E_{\pi}}[G_{t+1} | S_{t+1} = s', A_{t+1}=a'] = q^\pi(s', a')$, we can write:

$$ Part(2)= \sum_{s', a'} \; \pi(a'|s') ~ p(s'| s, a) q^\pi(s', a').$$

We then recognise that using the chain rule for the probabilities:

$$Part(2)=\sum_{s', a'} \; \pi(a'|s') \; p(s' | s, a) q^\pi(s', a') = \sum_{s', a'} \; p(s', a' | s, a) q^\pi(s', a')= \mathbb{E_{\pi}} [q^\pi(s', a')|S_t = s, A_t = a],$$

i.e., the conditional expectation of $q^\pi(s', a')$ across all possible $s'$ and $a'$ under the MDP and policy $\pi$.

We can finally place the parts back and use the linearity of expectations to combine to:

$$q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1} + \gamma \; q^\pi(s', a') | S_t = s, A_t = a].$$

We will use this expression to derive the Bellman equation for action-values.

**Note1:** Why $\pi(a'|s') p(s'| s, a) = p(s', a' | s, a)$? 

We recall that $\pi(a'|s')= p(a'|s')=p(a'|s',s,a)$, which follows from the Markovian property. We aim to show that:

$$p(a'|s',s,a) \; p(s'| s, a) = p(s', a' | s, a),$$

or in general:

$$p(A|B,C,D)\; p(B|C,D) = p(A, B|C,D).$$

From the definition of the conditional probabilities we have:

$$p(A|B,C,D) p(B|C,D) = \left(\frac{p(A \cap B \cap C \cap D)}{p(B \cap C \cap D)}\right) \left(\frac{p(B \cap C \cap D)}{p(C \cap D)}\right) = \frac{p((A \cap B) \cap (C \cap D))}{p(C \cap D)} = p(A, B|C,D).$$

**Note 2:** Why $\mathbb{E}_{\pi}[q^\pi(s', a')] = \sum_{s', a'} q^\pi(s', a') p(s', a' | s, a)$?

In reinforcement learning, $q^\pi(s', a')$, represents the expected return of the next state-action pair, starting from state $s$, taking action $a$, and following policy $\pi$. However, from a state $s$ and action $a$, multiple $s'$ states can be reached due to the stochasticity of the environment. For example, a robot in state $s$ may not always end up in the same state $s'$ when moving forward due to slipping. To account for this variability, we calculate the expectation on $q^\pi(s', a')$, considering all possible $s'$ states.

The expectation of a discrete variable $X$ conditioned to $Y$ is given by:

$$\mathbb{E}[X|Y=y] = \sum_{x} x P(X=x|Y=y).$$

In the reinforcement learning context, this translates to:

$$\mathbb{E}_{\pi}[q^\pi(s', a')| s,a] = \sum_{s', a'} q^\pi(s', a') p(s', a' | s, a),$$

where $X$ is parameterized by $q^\pi(s', a')$ representing returns, conditioned on $(s, a)$, since $s'$ and $a'$ are reached from $s$ and $a$ under policy $\pi$. The expectation considers all possible pairs of next states $s'$ and actions $a'$. Here, $p(s', a' | s, a)$ denotes the combined probability of transitioning to $s'$ and taking action $a'$ from $s$ with action $a$. 

**Note 3:** Why when calculating the expectation can I replace a variable $X$ with a parametrised function $Q(s',a')$ and take the sum over $s',a'$?

The Law of the Unconscious Statistician (LOTUS) allows us to compute the expectation of a function of a random variable without explicitly needing the function's probability distribution. Consider a random variable $X$ with a known probability distribution $P(X=x)$ and a function $g(X)$ that maps outcomes of $X$ to real numbers. According to LOTUS,  expected value of $g(X)$ is given by:

$$\mathbb{E}[g(X)] = \sum_{x} g(x) P(X=x),$$

We can then apply LOTUS, considering the state-action value $Q(s', a')$ instead of the function $g(X)$ parameterized by $s'$ and $a'$.  By extending the expression in the context of conditional probabilities, we get:

$$\mathbb{E}_{\pi}[q(s', a')] = \sum_{s', a'} q(s', a') p(s', a' | s, a),$$

where $p(s', a' | s, a)$ denotes the transition probability to $s', a'$ from $s, a$. 
For a formal proof of LOTUS see here: https://statproofbook.github.io/P/mean-lotus.html

## Example
<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/deterministic_mdp.png" width="800" />
</div>


Back to the analog safe scenario we discussed earlier. Opening requires turning the analog dial twice, first pointing to the dial $1$ and then to the dial $0.$ Reward R=1 is given upon opening the safe, for every other combination else the reward is zero. We will assume the dial has options for only two numbers, $0$ and $1$. How many state-action pairs are there? What are the expected state-action values of each state? The initial state is S0, the state where the agent lands after guessing the first digit correctly ($A_1=1$) is S1, and the state it lands if it guesses the first digit incorrectly ($A_1=0$) is S2. From S1, by guessing the second digit correctly ($A_2=0$) it arrives to $RT$ with R=1 and all other transitions get R=0.  Assume $\gamma=0.9$ and calculate $q^\pi(s, a)$ for a random policy (ever action is chosen with probability 50%).


We will work from the terminals upwards and make use of:

$$q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1}| S_t = s, A_t = a] + \gamma \mathbb{E_{\pi}}[q^\pi(s', a') | S_t = s, A_t = a] .$$

We have six q-values corresponding to the state action pairs $(s0,0), (s0,1), (s1,0), (s1,1), (s2,0), (s2,1)$. 

Terminal states  have no q-values (or they be considered 0). The only "rewarded" terminal is $TR$ so we will start with the $q(S,A2)$ and work bakwards. 

$$q^\pi(S=S1, A2=0) = \mathbb{E_{\pi}}[1 \; | s1, 0] + 0.9 \times 0=1.$$

The 0 here is because after taking action $A2$ we end up in a teminal state where there are no actions we can take and $\mathbb{E_{\pi}}[q^\pi(T,-) | S_t = s, A_t = a] =0$.

Similarily:

$$q^\pi(s1, A2=1) = \mathbb{E_{\pi}}[0 \; | s1, 1] + 0.9 \times 0=0.$$

$$q^\pi(s2, A2=0) = \mathbb{E_{\pi}}[0 \; | s2, 0] + 0.9 \times 0=0.$$

$$q^\pi(s2, A2=1) = \mathbb{E_{\pi}}[0 \; | s2, 1] + 0.9 \times 0=0.$$

We continue with the previous level, but now we need to consider the policy to decide how we will choose an action. To calculate $q^\pi(s1, A1=1)$, we need to consider the next state-action pair. Because the environment is deterministic, we know we will land on $s1$. However, we choose the next action $a'$ randomly. This means we should average between $q^\pi(s1, A2=0)$ and $q^\pi(s1, A2=1)$ to estimate $\mathbb{E_{\pi}}[q^\pi(s', a') | S_t = s, A_t = a]$:  

$$\mathbb{E_{\pi}}[q^\pi(s', a') | s0, 1]=\frac{q^\pi(s1, A2=0)+q^\pi(s1, 1)}{2}=0.5$$

$$q^\pi(s0, 1) = \mathbb{E_{\pi}}[0 \;| s0, 1]+ 0.9 \times 0.5=0.45$$

and

$$q^\pi(s0, 0) = \mathbb{E_{\pi}}[0 + 0.9 \times q^\pi(s2, 1) \;| s0, 1]=0.$$


# Exercise 

Still on the safe problem, now whenever you turn the dial to the correct digit, there is a $10\%$ probability that something jams and internally the dial becomes the incorrect digit. What are the expected state-action values in this case?

Use the following equation to compute the values:

$$q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1} + \gamma \; q^\pi(s', a') | S_t = s, A_t = a]= \mathbb{E_{\pi}}[R_{t+1}| S_t = s, A_t = a] + \gamma \mathbb{E_{\pi}}[q^\pi(s', a') | S_t = s, A_t = a] .$$

Calculate the values by working backward from the terminal state, noting that $q^\pi(s', a')=0$ for terminal states.

<details>
<summary>Show Solution</summary>

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/eleni-vasilaki/rl-notes/main/notebooks/stochastic_mdp.png" width="800" />
</div>

We will, again, make use of:

$$q^\pi(s, a) = \mathbb{E_{\pi}}[R_{t+1}| S_t = s, A_t = a] + \gamma \mathbb{E_{\pi}}[q^\pi(s', a') | S_t = s, A_t = a] .$$ 

The solution is nearly identical to the example with the difference that when the agent plays the correct action 10% of the time the system acts as if it has done the wrong action. 

As before, we have six q values: $(s0,0), (s0,1), (s1,0), (s1,1), (s2,0), (s2,1).$ However now expectations also differ.

For instance, for state $s1$ selecting the correct answer ($A2=0$), no longer gives reward 1 but only $90%$ of the time.

$\mathbb{E_{\pi}}[r \; | s1, 0]=0.9$

$$q^\pi(s1, A2=0) = 0.9  + 0.9 0=0.9$$

The rest remain the same apart from the state-action pairs involving $s0$.

$$q^\pi(s1, A2=1) =0.$$

$$q^\pi(s2, A2=0) =0.$$

$$q^\pi(s2, A2=1) =0.$$


Now when form $s1$ we take the correct action the reward is reduced in two ways.

First the expectation of the reward on the next state action pair is reduced in comparison to the stochastic environment. But also, though we are operating on a random policy, the split between doing the right action and doing the wrong action changes. Under random policy the split was equal, meaning that in $n$ attempts, we would have done $n/2$ correct actions and $n/2$ wrong actions. But out correct actions are now only interpreted as correct $0.9 \times\frac{ n}{2} $ while the missing $0.1 \times\frac{ n}{2}$ is as if we made an incorrect choise. Hence the probabilities become:

Correct: $0.9 \times\frac{n}{2n}=0.45$ and Incorrect: $0.1 \times\frac{ n}{2n}+\frac{ n}{2n}$

We then write:

$$\mathbb{E_{\pi}}[q^\pi(s', a') | s0, 1]=q^\pi(s1, A2=0)\times 0.55+q^\pi(s1, 1)\times 0.45= 0.9 \times 0.45=0.405$$

$$q^\pi(s0, 1) = \mathbb{E_{\pi}}[0 \;| s0, 1]+ 0.9 \times 0.405=0.36$$

while nothing chanhes for the non rewarded action.

$$q^\pi(s0, 0) = \mathbb{E_{\pi}}[0 + 0.9 \times q^\pi(s2, 1) \;| s0, 1]=0.$$
</details>

## Bellman Optimality Equation for Action-Value Functions

Our goal in RL (Reinforcement Learning) is to maximise the Reward. We therefore seek policies that will allow us to achieve as high returns as possible. We define an optimal policy as the policy that maximises the total return $G$ across all states and actions. This means that $\pi$ is an optimal policy if $q^\pi(s, a) \geq q^{\pi'}(s, a)$ for all state-action pairs. In other words:

$$q^*(s,a) = \max_{\pi} q^{\pi}(s,a).$$

Starting from the action-value equation that we derived earlier:

$$q^\pi(s, a) = \mathbb{E}_{\pi}[R_{t+1} + \gamma \; q^\pi(s', a') \; | \; S_t = s, A_t = a],$$

we can write the Bellman Optimality Equation for Action-Value Functions by selecting greedily the action $a'$ resulting in:

$$q^*(s, a) = \mathbb{E}[R_{t+1} + \gamma \max_{a'} q^*(s', a') \; | \; S_t = s, A_t = a].$$

This equation represents the expected return for taking an action $a$ in state $s$ and thereafter following the optimal policy. By iteratively applying this equation, we can converge to the optimal action-value function $q^*$, which in turn helps us to identify the optimal policy.


# Exercise

In the context of the bandit problem, when discussing different strategies, we highlighted the limitation that a greedy policy can lead to suboptimal solutions. The algorithmic implementation illustrates that the bandit is likely to choose more frequent but smaller rewards. Why, then, in the context of the Bellman Optimality Equation, does a greedy approach seem to be an optimal strategy?

<details>
<summary>Show Solution</summary>

A greedy policy in the bandit problem is suboptimal because the true q-values are not known . Hence, it only exploits the current best-known action, potentially missing better actions due to insufficient exploration. In contrast, the Bellman Optimality Equation assumes the true q-values are known, meaning the optimal strategy is greedy—always choosing the action with the highest expected return. The key difference is that in bandits, learning is still in progress, while in an MDP with known values, greediness is optimal.
</details>

## State Value Functions 

It is possible to also define the value function of a state, considering the expected reward across all actions that can be taken from that state. This is a more compressed form of information, typically followed in most textbooks. The value function, $v(s)$, can be defined as the expected return for being in a state $s$ and following policy $\pi$ thereafter. Formally, it is given by:

$$
v(s) = \mathbb{E}[G_t | S_t = s]
$$

where $G_t$ is the return at time $t$, and the expectation is taken over the distribution of possible paths that stem from following $\pi$, starting from $s$.

We can derive the Bellman equation for state values. The value of a state $s$ under a policy $\pi$, denoted $v_\pi(s)$, is the expected return starting from state $s$ and following policy $\pi$ thereafter. The Bellman equation for state values expresses this as:

$$
v_\pi(s) = \sum_{a} \pi(a|s) \sum_{s', r} p(s', r | s, a) \left[ r + \gamma v_\pi(s') \right]
$$

where:
- $\pi(a|s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$,
- $p(s', r | s, a)$ is the probability of transitioning to state $s'$ and receiving reward $r$ after taking action $a$ in state $s$,
- $\gamma$ is the discount factor, which represents the difference in importance between future rewards and present rewards.


## Optimal State Value Functions

It follows that the optimal state value function, denoted as $v^*(s)$, is defined as the maximum expected return achievable under any policy, from state $s$. Formally, it is given by $v^*(s) = \max_{a} q^{*}(s,a)$, where $q^{*}(s,a)$ represents the action-value function under the greedy policy. 

# Exercise 

Derive the Bellman equation for the state value functions. Explain all intermediate steps. Hint: Decompose it into two parts as in the proof of the Bellman equation for state action values, and work on them separately. For the first part, expand it by using the definition of expectation, and condition on  $a$ and $s'$ using the law of total expectation. Treat the second part similarly to the proof for the state-action values. To recombine the two parts, observe that the reward variable is absent in the expected probability expression in the second part. Incorporate it first, then marginalise it out.


<details>
<summary>Show Solution</summary>


### Proof of the Bellman Equation for State Values

#### Initial Setup

Given a state $s$ and a policy $\pi$, the value of $s$ under $\pi$ is defined as the expected return starting from $s$ and following $\pi$. The return $G_t$ is the sum of rewards received, discounted by $\gamma$ at each step into the future.

#### Step 1: Recursive Form of $G_t$

The return at time $t$ can be expressed recursively:
$$
G_t = R_{t+1} + \gamma G_{t+1}
$$

#### Step 2: Expected Return from State $s$

The value function $v_\pi(s)$ is the expected return starting from state $s$ and following policy $\pi$:
$$
v_\pi(s) = \mathbb{E}_\pi[G_t | S_t = s]
$$

#### Step 3: Decomposition Using Linearity of Expectation

Using the linearity of expectation, we decompose $v_\pi(s)$ into two components:
$$
v_\pi(s) = \mathbb{E}_\pi[R_{t+1} | S_t = s] + \gamma \mathbb{E}_\pi[G_{t+1} | S_t = s]
$$

#### Step 4: Immediate Reward Expectation

The expectation of the immediate reward $R_{t+1}$ is expressed and then expanded through conditioning on actions and next states:
$$
\mathbb{E}_\pi[R_{t+1} | S_t = s] = \sum_r r p(r|s)
$$
Conditioning on action $A_t$:
$$
\sum_a \pi(a|s) \sum_r r p(r|s,a)
$$
Further conditioning on the next state $S_{t+1}$ yields:
$$
\sum_a \pi(a|s) \sum_r \sum_{s'} r p(s'|s,a) p(r|s,a,s')
$$
Recognizing that $p(s'|s,a) p(r|s,a,s') = p(r,s'|s,a)$ simplifies to:
$$
\sum_a \pi(a|s) \sum_{s', r} r p(r,s'|s,a)
$$

#### Step 5: Future Rewards Expectation

We would like to condition the term:

$$\gamma \mathbb{E}_\pi[G_{t+1} | S_t = s]$$

to the future state $s'$ because $ v_\pi(s')=\mathbb{E}_\pi[G_{t+1} | S_{t+1} = s']$. Since we end up at state $s'$ via action $a$ we will first condition to the action $a$,  while applying the law of total expectation:

$$\sum_a \pi(a|s) \gamma \mathbb{E}_\pi[G_{t+1} | S_t = s, A_t=a]$$

Then we condition on $S_{t+1}$, again applying the law of total expectation:

$$\sum_a \pi(a|s) \sum_s' p(s'|s,a) \gamma \mathbb{E}_\pi[G_{t+1} | S_t = s, A_t=a, S{t+1}=s']$$

Further, we leverage the Markov property, thus $\mathbb{E}_\pi[G_{t+1} | S_t = s, A_t=a, S_{t+1}=s']=\mathbb{E}_\pi[G_{t+1} | S_{t+1}=s']$ and we obtain:
$$
\gamma \sum_a \pi(a|s) \sum_{s'} p(s'|s,a) v_\pi(s')
$$

Because we would like to combine this part with the immediate reward, we will make use of marginalisation introducing the reward, and summing across all values $r$:

$$
\gamma \sum_a \pi(a|s) \sum_{s',r}  p(s',r|s,a) v_\pi(s')
$$

#### Step 6: Combine Immediate and Future Rewards

Integrating both parts and treating the conditional probabilities jointly:
$$
v_\pi(s) = \sum_a \pi(a|s) \sum_{s', r} p(r,s'|s,a) [r + \gamma v_\pi(s')]
$$

This completes the proof of the Bellman equation for state values under policy $\pi$, illustrating how the expected sum of immediate rewards and discounted future rewards is computed.

</details>
